# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [16]:
import pandas as pd
import numpy as np
import re


def load_data():
    messages = pd.read_csv("./data/disaster_messages.csv")
    categories = pd.read_csv("./data/disaster_categories.csv")

    return messages, categories


def transform_category(df_category: pd.DataFrame):
    df_category_expanded = df_category["categories"].str.split(";", expand=True)

    def _extract_label_names(record):
        _label_names = [re.sub('-[0-9]+', "", col) for col in record]
        return _label_names

    def _extract_label_values(series: pd.Series, label_name):
        return series.apply(lambda c: c.replace(f"{label_name}-", ""))

    # Get label names and set new columns
    label_names = _extract_label_names(df_category_expanded.iloc[0])
    df_category_expanded.columns = label_names

    # Convert label values of each series into numberic
    for label_name in label_names:
        df_category_expanded[label_name] = _extract_label_values(df_category_expanded[label_name], label_name)


    return pd.concat([df_category.drop(labels=['categories'], axis=1), df_category_expanded], axis=1)


def merge_dataset(df_message, df_category):
    return pd.merge(df_message, df_category, on=['id'])


messages, categories = load_data()


messages_merged = merge_dataset(messages, categories)

messages_merged_trans = transform_category(messages_merged)

messages_merged_trans


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26381,30261,The training demonstrated how to enhance micro...,NaN,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26382,30262,A suitable candidate has been selected and OCH...,NaN,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26383,30263,"Proshika, operating in Cox's Bazar municipalit...",NaN,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26384,30264,"Some 2,000 women protesting against the conduc...",NaN,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
def check_duplicated(_df: pd.DataFrame):
    duplicate_rows = messages_merged_trans[messages_merged_trans.duplicated()]
    print(f'Found {len(duplicate_rows.index)} duplicated values...')
    print('We will removed these duplicated values...')

    messages_merged_trans.drop_duplicates(inplace=True)




In [18]:
from sqlalchemy import create_engine
# engine = create_engine('sqlite://', echo=False)

def save_to_db(_df: pd.DataFrame, database_name, table_name):
    engine = create_engine(f'sqlite:///{database_name}')
    _df.to_sql(table_name, engine, index=False)

In [19]:
check_duplicated(messages_merged_trans)

save_to_db(messages_merged_trans, 'disaster.db', 'message_category')

Found 170 duplicated values...
We will removed these duplicated values...
